In [28]:
import pandas as pd
import pulp
import os
from functools import partial

In [29]:
os.chdir("C:/Users/batuh/Documents/GitHub/doktar-opt/ExampleLPPProgram/scenarios/scenario-1")

In [30]:
base_data = pd.read_csv("./variable-coefficients/base/DoktarVariableBase_81.csv")
print(base_data.head(5))
print(base_data.shape[0])
print(base_data.shape[1])


   CityId  DistrictId  IrrigationId  FieldId CityName DistrictName  FieldArea  \
0      81         846             1  2068001    DUZCE      GOLYAKA   0.277168   
1      81         854             0  2068002    DUZCE     KAYNASLI   0.439520   
2      81         863             1  2068003    DUZCE     GUMUSOVA   0.427013   
3      81         888             0  2068004    DUZCE      CILIMLI   0.274361   
4      81         901             1  2068005    DUZCE     CUMAYERI   0.694518   

                                            geometry  WheatPercent  \
0  MULTIPOLYGON (((30.91682406996663 40.728429080...          1.15   
1  MULTIPOLYGON (((30.91685332263513 40.727393715...          1.05   
2  MULTIPOLYGON (((30.9195760769791 40.7293344734...          1.15   
3  MULTIPOLYGON (((30.98546681716249 40.736468600...          1.05   
4  MULTIPOLYGON (((30.93162598297895 40.645469781...          1.09   

   BarleyPercent  ...  SugarbeetPercent  SoybeanPercent  PeanutPercent  \
0           1.15  

In [31]:
yield_data = pd.read_csv("./variable-coefficients/yield/DoktarVariableYield_scenario-1.csv")
yield_data.head(5)

,CityId,DistrictId,CityName,DistrictName,WheatYield,BarleyYield,OatYield,CottonYield,SoybeanYield,TomatoYield,...,PotatoYield,SugarbeetYield,OnionYield,BeanYield,LentilYield,ChickpeaYield,PeanutYield,RiceYield,CornYield,SunflowerYield
0,31,15,HATAY,YAYLADA?I,292,265,278,494,442,7244,...,3514,6846,3329,271,157,123,394,782,941,292
1,31,17,HATAY,ALTIN�Z�,292,265,278,494,442,7244,...,3514,6846,3329,271,157,123,394,782,941,292
2,31,20,HATAY,DEFNE,292,265,278,494,442,7244,...,3514,6846,3329,271,157,123,394,782,941,292
3,33,25,MERS?N,AYDINCIK,292,265,278,494,442,7244,...,3514,6846,3329,271,157,123,394,782,941,292
4,31,26,HATAY,SAMANDA?,292,265,278,494,442,7244,...,3514,6846,3329,271,157,123,394,782,941,292


In [32]:
price_data = pd.read_csv("./variable-coefficients/price/DoktarVariablePrice_scenario-1.csv")
price_data.head(5)

,CityId,DistrictId,CityName,DistrictName,WheatPrice,BarleyPrice,OatPrice,CottonPrice,SoybeanPrice,TomatoPrice,...,PotatoPrice,SugarbeetPrice,OnionPrice,BeanPrice,LentilPrice,ChickpeaPrice,PeanutPrice,RicePrice,CornPrice,SunflowerPrice
0,31,15,HATAY,YAYLADA?I,2.06,1.62,2.1,9.59,2.18,0.55,...,1.5,0.32,2.8,7.98,4.85,4.9,10.4,4.4,1.4,6.11
1,31,15,HATAY,YAYLADA?I,2.06,1.62,2.1,9.59,2.18,0.55,...,1.5,0.32,2.8,7.98,4.85,4.9,10.4,4.4,1.4,6.11
2,31,17,HATAY,ALTIN�Z�,2.06,1.62,2.1,9.59,2.18,0.55,...,1.5,0.32,2.8,7.98,4.85,4.9,10.4,4.4,1.4,6.11
3,31,20,HATAY,DEFNE,2.06,1.62,2.1,9.59,2.18,0.55,...,1.5,0.32,2.8,7.98,4.85,4.9,10.4,4.4,1.4,6.11
4,33,25,MERS?N,AYDINCIK,2.06,1.62,2.1,9.59,2.18,0.55,...,1.5,0.32,2.8,7.98,4.85,4.9,10.4,4.4,1.4,6.11


In [33]:
cost_data = pd.read_csv("./variable-coefficients/cost/DoktarVariableCost_scenario-1.csv")
cost_data.head(5)

,CityId,DistrictId,IrrigationId,CityName,DistrictName,WheatCost,BarleyCost,OatCost,CottonCost,SoybeanCost,...,PotatoCost,SugarbeetCost,OnionCost,BeanCost,LentilCost,ChickpeaCost,PeanutCost,RiceCost,CornCost,SunflowerCost
0,31,15,1,HATAY,YAYLADA?I,798,773,763,1823,1268,...,3656,1879,3378,1302,616,597,2495,2593,1391,909
1,31,15,1,HATAY,YAYLADA?I,798,773,763,1823,1268,...,3656,1879,3378,1302,616,597,2495,2593,1391,909
2,31,17,1,HATAY,ALTIN�Z�,798,773,763,1823,1268,...,3656,1879,3378,1302,616,597,2495,2593,1391,909
3,31,20,1,HATAY,DEFNE,798,773,763,1823,1268,...,3656,1879,3378,1302,616,597,2495,2593,1391,909
4,33,25,1,MERS?N,AYDINCIK,798,773,763,1823,1268,...,3656,1879,3378,1302,616,597,2495,2593,1391,909


In [34]:
def add_yield(base_data, yield_data):
    base_data = pd.merge(base_data, yield_data, on = "DistrictId", how = "left", suffixes = ("", "_extra"))
    base_data= base_data[base_data.columns.drop(list(base_data.filter(regex='_extra')))]
    return base_data


def add_price(base_data, price_data):
    base_data = pd.merge(base_data, price_data, on = "DistrictId", how = "left", suffixes = ("", "_extra"))
    base_data= base_data[base_data.columns.drop(list(base_data.filter(regex='_extra')))]
    return base_data


def add_cost(base_data, cost_data):
    base_data = pd.merge(base_data, cost_data, on = ['DistrictId', 'IrrigationId'], how = 'left', suffixes = ('', '_extra'))
    base_data= base_data[base_data.columns.drop(list(base_data.filter(regex='_extra')))]
    return base_data

In [35]:
print(base_data.shape[0])
print(base_data.shape[1])

6674
25


In [36]:
base_data = add_yield(base_data, yield_data)
base_data = add_price(base_data, price_data)
base_data = add_cost(base_data, cost_data)

In [37]:
print(base_data.shape[0])
print(base_data.shape[1])

6674
76


In [25]:
def wide_to_long(crop_name, base_data):
    main_columns = ['CityId', 'DistrictId', 'FieldId', 'IrrigationId', 'CityName', 'DistrictName', 'FieldArea', 'geometry']
    crop_columns = list(base_data.filter(regex=(crop_name)).columns)
    base_data = base_data.filter(main_columns + crop_columns)
    base_data = base_data.rename(columns={crop_name + 'Percent': 'Percent', crop_name + 'Yield': 'Yield', crop_name + 'Price': 'Price', crop_name + 'Cost': 'Cost'})
    base_data['CropName'] = crop_name
    return base_data

In [26]:
crop_list = ['Wheat', 'Barley', 'Oat', 'Cotton', 'Soybean', 'Tomato', 'Canola', 'Potato', 'Sugarbeet', 'Onion', 'Bean', 'Lentil', 'Chickpea', 'Peanut', 'Rice', 'Corn', 'Sunflower']

wide_func = partial(wide_to_long, base_data = base_data)
long_list = list(map(wide_func, crop_list))
long_data = pd.concat(long_list)
long_data.head(5)
print(long_data.shape[0])
print(long_data.shape[1])

113458
13


In [27]:
long_data

,CityId,DistrictId,FieldId,IrrigationId,CityName,DistrictName,FieldArea,geometry,Percent,Yield,Price,Cost,CropName
0,81,846,2068001,1,DUZCE,GOLYAKA,0.277168,MULTIPOLYGON (((30.91682406996663 40.728429080...,1.15,292,2.06,798,Wheat
1,81,854,2068002,0,DUZCE,KAYNASLI,0.439520,MULTIPOLYGON (((30.91685332263513 40.727393715...,1.05,292,2.06,458,Wheat
2,81,863,2068003,1,DUZCE,GUMUSOVA,0.427013,MULTIPOLYGON (((30.9195760769791 40.7293344734...,1.15,292,2.06,798,Wheat
3,81,888,2068004,0,DUZCE,CILIMLI,0.274361,MULTIPOLYGON (((30.98546681716249 40.736468600...,1.05,292,2.06,458,Wheat
4,81,901,2068005,1,DUZCE,CUMAYERI,0.694518,MULTIPOLYGON (((30.93162598297895 40.645469781...,1.09,292,2.06,798,Wheat
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6669,81,901,2074670,0,DUZCE,CUMAYERI,0.453471,MULTIPOLYGON (((31.11097228702912 40.850215072...,1.00,292,6.11,473,Sunflower
6670,81,888,2074671,0,DUZCE,CILIMLI,0.685165,MULTIPOLYGON (((31.11967166708359 40.890230105...,0.90,292,6.11,473,Sunflower
6671,81,863,2074672,1,DUZCE,GUMUSOVA,3.841589,MULTIPOLYGON (((31.13569309040149 40.880117033...,0.90,292,6.11,909,Sunflower
6672,81,863,2074673,0,DUZCE,GUMUSOVA,0.628193,MULTIPOLYGON (((31.2034339569929 40.8291888400...,0.90,292,6.11,473,Sunflower


### TESTING ------

In [ ]:
test_dat = pd.merge(base_data, yield_data, on = "DistrictId", how = "left", suffixes = ("", "_extra"))
test_dat= test_dat[test_dat.columns.drop(list(test_dat.filter(regex='_extra')))]
test_dat.head(5)
print(test_dat.columns)
print(base_data.shape[0])
print(test_dat.shape[0])
print(base_data.shape[1])
print(test_dat.shape[1])

In [26]:
    # Scenario List
    scenario_list = ['scenario-1','scenario-2','scenario-3','scenario-4','scenario-5']
    coef_list = ['Revenue', 'Revenue', 'Physical', 'Revenue', 'Revenue']
    scenario_name = 'scenario-1'
    # Scenario Dict
    sc_dict = {scenario_list[i]: coef_list[i] for i in range(len(scenario_list))} 
    sc_dict.get(scenario_name)

'Revenue'